# RAG with LlamaIndex and AstraDB

Build a RAG pipeline with RAGStack, AstraDB, and LlamaIndex.

## Prerequisites
You will need a vector-enabled Astra database.

Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
Get your Astra DB Endpoint:
https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup

In [ ]:
!pip install -qU ragstack-ai

In [ ]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
keys = ["ASTRA_DB_APPLICATION_TOKEN", "ASTRA_DB_API_ENDPOINT", "OPENAI_API_KEY"]
for key in keys:
    if key not in os.environ:
        os.environ[key] = getpass(f"Enter {key}: ")

In [ ]:
# Collections are where documents are stored. ex: test
collection = input("Collection: ")

## Create RAG pipeline

### Embedding model and vector store

Load a sample dataset from Llama Hub into your Astra vector store.

In [ ]:
from llama_index.llama_dataset import download_llama_dataset

!mkdir -p 'data'

dataset = download_llama_dataset(
  "PaulGrahamEssayDataset", "./data"
)

In [ ]:
from llama_index.vector_stores import AstraDBVectorStore
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)

documents = SimpleDirectoryReader("./data").load_data()
print(f"Total documents: {len(documents)}")
print(f"First document, id: {documents[0].doc_id}")
print(f"First document, hash: {documents[0].hash}")
print(
    "First document, text"
    f" ({len(documents[0].text)} characters):\n"
    f"{'=' * 20}\n"
    f"{documents[0].text[:360]} ..."
)

astra_db_store = AstraDBVectorStore(
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    collection_name="test",
    embedding_dimension=1536,
)

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### Query

Query the index for the most relevant answer to your prompt, "Why did the author choose to work on AI?"

In [ ]:
query_engine = index.as_query_engine()
query_string_1 = "Why did the author choose to work on AI?"
response = query_engine.query(query_string_1)

print(query_string_1)
print(response.response)

Use a retriever to retrieve results from your vector store index based on your prompt.

This will retrieve three nodes based on your prompt, and return the nodes with their relevance scores.

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode="default",
    similarity_top_k=3,
)

nodes_with_scores = retriever.retrieve(query_string_1)

print(query_string_1)
print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:90]} ...")

### MMR

Set the retriever to sort results by Maximal Marginal Relevance, or MMR, instead of the default similarity search.

Send the prompt again. The top result is the most relevant (positive number), while the other results are the least relevant (negative numbers).

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_prefetch_factor": 4},
)

nodes_with_scores = retriever.retrieve(query_string_1)

print(query_string_1)
print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:90]} ...")